In [ ]:
pip install opencv-python


In [ ]:
import cv2
import os
import numpy as np

def extract_10_frames(video_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames - 1, 10, dtype=int)

    count = 0
    saved = 0
    for idx in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        if idx in frame_indices:
            frame_name = f"frame_{saved:02d}.jpg"
            frame_path = os.path.join(output_dir, frame_name)
            cv2.imwrite(frame_path, frame)
            saved += 1
        count += 1

    cap.release()
    print(f"✅ Saved {saved} frames to: {output_dir}")

# ✅ Use raw strings to avoid \ errors
video_path = r"/content/drive/MyDrive/Research/FT3-LA.mp4"
output_dir = r"/content/drive/MyDrive/Research/Combined Data/Frames"

extract_10_frames(video_path, output_dir)


✅ Saved 10 frames to: /content/drive/MyDrive/Research/Combined Data/Frames


In [ ]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
from ultralytics import YOLO
import cv2
import os

def annotate_and_save_labels(frames_folder, output_folder, label_folder, yolo_model_path='yolov8n.pt', conf_thresh=0.3):
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(label_folder, exist_ok=True)

    model = YOLO(yolo_model_path)
    frame_files = sorted([f for f in os.listdir(frames_folder) if f.endswith('.jpg')])

    for file in frame_files:
        img_path = os.path.join(frames_folder, file)
        img = cv2.imread(img_path)
        height, width = img.shape[:2]

        # Run YOLOv8 inference
        results = model(img)[0]

        best_box = None
        best_conf = 0.0

        # Find the most confident 'person' (class 0)
        for box in results.boxes:
            cls_id = int(box.cls)
            conf = float(box.conf)
            if cls_id == 0 and conf > conf_thresh and conf > best_conf:
                best_box = box
                best_conf = conf

        label_lines = []

        if best_box:
            x1, y1, x2, y2 = map(float, best_box.xyxy[0])

            # Convert to YOLO format (normalized)
            x_center = (x1 + x2) / 2 / width
            y_center = (y1 + y2) / 2 / height
            box_width = (x2 - x1) / width
            box_height = (y2 - y1) / height

            label_lines.append(f"0 {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}")

            # Draw the bounding box and label
            cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(img, f"Athlete {best_conf:.2f}", (int(x1), int(y1) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Save annotated image
        annotated_img_path = os.path.join(output_folder, file)
        cv2.imwrite(annotated_img_path, img)

        # Save label file (same name but .txt)
        label_filename = os.path.splitext(file)[0] + ".txt"
        label_path = os.path.join(label_folder, label_filename)
        with open(label_path, 'w') as f:
            f.write('\n'.join(label_lines))

    print(f"✅ Annotated only the athlete in {len(frame_files)} frames.")
    print(f"🖼️ Annotated images → {output_folder}")
    print(f"📝 YOLO .txt labels → {label_folder}")

# Example usage:
frames_folder = r"/content/drive/MyDrive/Research/Combined Data/Frames"
output_folder = r"/content/drive/MyDrive/Research/Combined Data/Frames/YOLO/FrameAnnotated"
label_folder = r"/content/drive/MyDrive/Research/Combined Data/Frames/YOLO/FrameLabels"
yolo_model_path = "yolov8n.pt"  # or your custom model path

annotate_and_save_labels(frames_folder, output_folder, label_folder, yolo_model_path)



0: 384x640 6 persons, 132.0ms
Speed: 2.8ms preprocess, 132.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 143.0ms
Speed: 4.0ms preprocess, 143.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 133.7ms
Speed: 3.5ms preprocess, 133.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 129.8ms
Speed: 3.8ms preprocess, 129.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 134.8ms
Speed: 4.1ms preprocess, 134.8ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 1 sports ball, 1 tennis racket, 128.0ms
Speed: 3.0ms preprocess, 128.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 130.1ms
Speed: 3.7ms preprocess, 130.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 145.1ms
Speed: 3.3ms preprocess, 145.1ms 

In [ ]:
from ultralytics import YOLO
import cv2
import os

def annotate_and_save_labels(frames_folder, output_folder, label_folder, yolo_model_path='yolov8n.pt', conf_thresh=0.25):
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(label_folder, exist_ok=True)

    # Load YOLOv8 model (pretrained or custom)
    model = YOLO(yolo_model_path)

    # Collect all image files
    frame_files = sorted([f for f in os.listdir(frames_folder) if f.endswith('.jpg') or f.endswith('.png')])

    total_labels = 0

    for file in frame_files:
        img_path = os.path.join(frames_folder, file)
        img = cv2.imread(img_path)
        if img is None:
            print(f"⚠️ Could not read image: {file}")
            continue

        height, width = img.shape[:2]
        results = model(img)[0]  # Run inference

        best_box = None
        best_conf = 0.0

        # Loop through detections
        for box in results.boxes:
            cls_id = int(box.cls)
            conf = float(box.conf)

            if cls_id == 0 and conf > conf_thresh and conf > best_conf:
                best_box = box
                best_conf = conf

        label_lines = []

        if best_box:
            x1, y1, x2, y2 = map(float, best_box.xyxy[0].tolist())

            # Convert to YOLO format
            x_center = (x1 + x2) / 2 / width
            y_center = (y1 + y2) / 2 / height
            box_width = (x2 - x1) / width
            box_height = (y2 - y1) / height

            label_lines.append(f"0 {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}")

            # Draw bounding box
            cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(img, f"Athlete {best_conf:.2f}", (int(x1), int(y1) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Save annotated image
        annotated_img_path = os.path.join(output_folder, file)
        cv2.imwrite(annotated_img_path, img)

        # Save label file only if person was found
        if label_lines:
            label_filename = os.path.splitext(file)[0] + ".txt"
            label_path = os.path.join(label_folder, label_filename)
            with open(label_path, 'w') as f:
                f.write('\n'.join(label_lines))
            total_labels += 1
            print(f"✅ {file}: Athlete detected (conf={best_conf:.2f}), label saved.")
        else:
            print(f"❌ {file}: No person detected above threshold.")

    print(f"\n✅ Done: {total_labels} athlete labels created.")
    print(f"🖼️ Annotated images → {output_folder}")
    print(f"📝 YOLO .txt labels → {label_folder}")

# --- Example Usage ---
frames_folder = r"/content/drive/MyDrive/Research/Combined Data/Frames"
output_folder = r"/content/drive/MyDrive/Research/Combined Data/Frames/YOLO/FrameAnnotated"
label_folder = r"/content/drive/MyDrive/Research/Combined Data/Frames/YOLO/FrameLabels"
yolo_model_path = "yolov8n.pt"  # or your custom-trained .pt

annotate_and_save_labels(frames_folder, output_folder, label_folder, yolo_model_path)



0: 384x640 6 persons, 142.5ms
Speed: 5.1ms preprocess, 142.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
✅ frame_00.jpg: Athlete detected (conf=0.77), label saved.

0: 384x640 4 persons, 128.3ms
Speed: 3.7ms preprocess, 128.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
✅ frame_01.jpg: Athlete detected (conf=0.81), label saved.

0: 384x640 5 persons, 140.1ms
Speed: 3.6ms preprocess, 140.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
✅ frame_02.jpg: Athlete detected (conf=0.83), label saved.

0: 384x640 4 persons, 141.6ms
Speed: 3.8ms preprocess, 141.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
✅ frame_03.jpg: Athlete detected (conf=0.87), label saved.

0: 384x640 5 persons, 143.6ms
Speed: 3.1ms preprocess, 143.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
✅ frame_04.jpg: Athlete detected (conf=0.91), label saved.

0: 384x640 6 persons, 1 sports ball, 1 tennis racket, 127.0

In [ ]:
pip install mediapipe opencv-python numpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1

In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp

mp_pose = mp.solutions.pose

# --- Utility: Calculate joint angle ---
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cos_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-8)
    angle = np.arccos(np.clip(cos_angle, -1.0, 1.0))
    return np.degrees(angle)

# --- Extract features from one frame using YOLO label ---
def extract_features_from_frame(image, label_path, width, height):
    if not os.path.exists(label_path):
        return None

    with open(label_path, 'r') as f:
        line = f.readline().strip()
        if not line:
            return None
        _, x_c, y_c, w, h = map(float, line.split())

    x1 = int((x_c - w / 2) * width)
    y1 = int((y_c - h / 2) * height)
    x2 = int((x_c + w / 2) * width)
    y2 = int((y_c + h / 2) * height)
    x1, y1 = max(x1, 0), max(y1, 0)
    x2, y2 = min(x2, width), min(y2, height)

    cropped = image[y1:y2, x1:x2]

    # Apply MediaPipe Pose
    with mp_pose.Pose(static_image_mode=True, model_complexity=1) as pose:
        results = pose.process(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))
        if not results.pose_landmarks:
            return None

        features = []
        for lm in results.pose_landmarks.landmark:
            abs_x = (lm.x * (x2 - x1) + x1) / width
            abs_y = (lm.y * (y2 - y1) + y1) / height
            abs_z = lm.z
            vis = lm.visibility
            features.extend([abs_x, abs_y, abs_z, vis])  # 33 × 4 = 132

        def coord(idx):
            return [
                results.pose_landmarks.landmark[idx].x * (x2 - x1) + x1,
                results.pose_landmarks.landmark[idx].y * (y2 - y1) + y1,
            ]

        try:
            angles = [
                calculate_angle(coord(11), coord(13), coord(15)),  # left elbow
                calculate_angle(coord(12), coord(14), coord(16)),  # right elbow
                calculate_angle(coord(23), coord(25), coord(27)),  # left knee
                calculate_angle(coord(24), coord(26), coord(28)),  # right knee
                calculate_angle(coord(11), coord(12), coord(14)),  # left shoulder
                calculate_angle(coord(12), coord(11), coord(13)),  # right shoulder
            ]
        except:
            angles = [0] * 6

        features.extend(angles)  # Total = 138
        return features

# --- Process all frames in groups of 10 (with padding if needed) ---
def process_frames_for_prediction(frame_folder, label_folder, save_path_x):
    frame_files = sorted([f for f in os.listdir(frame_folder) if f.endswith('.jpg') or f.endswith('.png')])
    total_frames = len(frame_files)
    sample_features = []

    for idx, frame_file in enumerate(frame_files):
        image_path = os.path.join(frame_folder, frame_file)
        label_path = os.path.join(label_folder, os.path.splitext(frame_file)[0] + ".txt")

        image = cv2.imread(image_path)
        if image is None:
            print(f"⚠️ Cannot read {frame_file}, skipping.")
            continue

        h, w = image.shape[:2]
        features = extract_features_from_frame(image, label_path, w, h)
        if features is None:
            print(f"⚠️ No pose for {frame_file}, skipping.")
            continue

        sample_features.append(features)

    if len(sample_features) == 0:
        print("❌ No valid frames for prediction.")
        return

    # Pad to 10 frames if needed
    while len(sample_features) < 10:
        sample_features.append(sample_features[-1])  # repeat last frame

    # Trim if more than 10 (optional)
    sample_features = sample_features[:10]

    data = np.array(sample_features)  # shape: (10, 138)
    velocities = np.gradient(data[:, :99], axis=0)
    enhanced = np.concatenate([data, velocities], axis=1)  # (10, 237)

    X_pred = np.expand_dims(enhanced, axis=0)  # shape: (1, 10, 237)
    np.save(save_path_x, X_pred)

    print(f"\n✅ Saved 1 padded prediction sample.")
    print(f"Shape: {X_pred.shape}")
    print(f"📁 File saved to: {save_path_x}")

# --- CONFIG ---
frame_folder = r"/content/drive/MyDrive/Research/Combined Data/Frames/YOLO/FrameAnnotated"
label_folder = r"/content/drive/MyDrive/Research/Combined Data/Frames/YOLO/FrameLabels"
save_path_x = r"/content/drive/MyDrive/Research/Combined Data/Frames/YOLO/X_pred.npy"

# --- RUN ---
process_frames_for_prediction(frame_folder, label_folder, save_path_x)



✅ Saved 1 padded prediction sample.
Shape: (1, 10, 237)
📁 File saved to: /content/drive/MyDrive/Research/Combined Data/Frames/YOLO/X_pred.npy


In [ ]:
pip install joblib


In [ ]:
import joblib

scaler = joblib.load("/content/drive/MyDrive/Research/New Updated Model/Model/Improved_scaler.save")
X_pred_raw = np.load("/content/drive/MyDrive/Research/Combined Data/Frames/YOLO/X_pred.npy")  # shape (1, 10, 237)

# Flatten, scale, reshape
X_pred_scaled = scaler.transform(X_pred_raw.reshape(-1, X_pred_raw.shape[-1]))
X_pred_scaled = X_pred_scaled.reshape(X_pred_raw.shape)

# Then predict
prediction = model.predict(X_pred_scaled)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step


In [ ]:
import joblib
import numpy as np
import tensorflow as tf

# Load model
model = tf.keras.models.load_model("/content/drive/MyDrive/Research/New Updated Model/Model/Improved_best_model.keras", safe_mode=False)

# Load scaler
scaler = joblib.load("/content/drive/MyDrive/Research/New Updated Model/Model/Improved_scaler.save")

# Load input
X_pred = np.load("/content/drive/MyDrive/Research/Combined Data/Frames/YOLO/X_pred.npy")

# Apply scaler frame-wise
original_shape = X_pred.shape  # (1, 10, 237)
X_reshaped = X_pred.reshape(-1, original_shape[-1])  # (10, 237)
X_scaled = scaler.transform(X_reshaped)
X_scaled = X_scaled.reshape(original_shape)  # (1, 10, 237)

# Predict
pred = model.predict(X_scaled)
pred_class = np.argmax(pred[0])
confidence = np.max(pred[0])

class_map = {
    0: "Good Technique",
    1: "Low Arm",
    2: "Poor Left Leg Block",
    3: "Both Errors"
}

print(f"\n🎯 Predicted Class: {pred_class} ({class_map[pred_class]})")
print(f"📊 Confidence: {confidence:.4f}")

TypeError: <class 'keras.src.models.functional.Functional'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'keras.src.models.functional', 'class_name': 'Functional', 'config': {}, 'registered_name': 'Functional', 'build_config': {'input_shape': None}, 'compile_config': {'optimizer': {'module': 'keras.optimizers', 'class_name': 'Adam', 'config': {'name': 'adam', 'learning_rate': 6.25000029685907e-05, 'weight_decay': None, 'clipnorm': None, 'global_clipnorm': None, 'clipvalue': None, 'use_ema': False, 'ema_momentum': 0.99, 'ema_overwrite_frequency': None, 'loss_scale_factor': None, 'gradient_accumulation_steps': None, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}, 'registered_name': None}, 'loss': 'sparse_categorical_crossentropy', 'loss_weights': None, 'metrics': ['accuracy'], 'we

In [ ]:
import numpy as np

# Load model and input
model = tf.keras.models.load_model("/content/drive/MyDrive/Research/New Updated Model/Model/New_best_model.keras")
X_pred = np.load("/content/drive/MyDrive/Research/Combined Data/Frames/YOLO/X_pred.npy")

# Print raw model output (before argmax)
pred = model.predict(X_pred)
print("🔢 Softmax Output:", pred[0])
print("🎯 Predicted class:", np.argmax(pred[0]))


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
🔢 Softmax Output: [ 0.00015074   0.0026037    0.029255     0.96799]
🎯 Predicted class: 3


In [ ]:
X_pred = np.load("/content/drive/MyDrive/Research/Combined Data/Frames/YOLO/X_pred.npy")  # shape should be (N, 10, 237)

for i in range(len(X_pred)):
    pred = model.predict(X_pred[i:i+1])
    print(f"Sample {i+1}: {pred[0]} → Predicted: {np.argmax(pred[0])}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Sample 1: [ 0.00015074   0.0026037    0.029255     0.96799] → Predicted: 3


In [ ]:
import numpy as np
X_pred = np.load("/content/drive/MyDrive/Research/Combined Data/Frames/YOLO/X_pred.npy")
print("Shape of prediction input:", X_pred.shape)


Shape of prediction input: (1, 10, 237)


In [ ]:
for i in range(X_pred.shape[0]):
    pred = model.predict(X_pred[i:i+1])
    print(f"Sample {i+1}: {pred[0]} → Predicted class: {np.argmax(pred[0])}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Sample 1: [ 0.00015074   0.0026037    0.029255     0.96799] → Predicted class: 3


In [ ]:
from sklearn.metrics import classification_report

y_true = [...]  # true test labels
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

print(classification_report(y_true, y_pred_labels, target_names=["Good", "Low Arm", "Poor Leg", "Both Errors"]))


NameError: name 'X_test' is not defined